In [1]:
"""!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install transformers kobert-transformers"""

'!pip install transformers\n!pip install torch\n!pip install scikit-learn\n!pip install transformers kobert-transformers'

In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [3]:
import pandas as pd

from transformers import BertTokenizerFast, DistilBertForSequenceClassification#model, tokenizer

In [4]:
!pip install wandb
import wandb

wandb.login()

wandb: Currently logged in as: sallylee359 (sallylee). Use `wandb login --relogin` to force relogin


True

데이터 처리

In [5]:
train_df = pd.read_csv('/content/gdrive/MyDrive/dataset/train_data.csv')
vaild_df = pd.read_csv('/content/gdrive/MyDrive/dataset/validation_data.csv')

In [6]:
from sklearn.preprocessing import MultiLabelBinarizer

train_df['empathy'] = train_df['empathy'].apply(lambda x:list(map(int,str(x).split(','))))
mlb = MultiLabelBinarizer()
y_train = mlb.fit_transform(train_df['empathy'])
X_train = train_df['text'].tolist()

In [7]:
X_train

['엄마, 아기가 태어나니까 내가 부모로서 해야 할 게 참 많은 것 같아요.',
 '그렇지? 아기 키우는 게 여간 어려운 일이 아니야.',
 '어제 평소보다도 격하게 막 온몸을 써가면서 울더라고요. 얼마나 당황했는지 몰라요.',
 '배가 고파서 그랬던 것 아닐까? 아기들은 배가 고프면 몸부림을 친단다.',
 '맞아요. 젖을 물려주니 금세 뚝 그쳤어요. 난 분명히 순했을 것 같은데.',
 '말도 마. 네가 얼마나 까탈스러웠는데. 우리 손주가 내 딸을 빼다 박았네.',
 '잠도 못 자고 우는 아이 달래랴 수유하랴 머리카락이 다 빠지는 줄 알았어요.',
 '네가 어른 노릇, 부모 노릇을 하느라 고생이 정말 많구나.',
 '그러면서 엄마 생각이 많이 났어요. 엄마는 직장까지 다니면서 나를 키우느라 얼마나 힘들었을까 너무 감사한 마음이 들더라고요.',
 '우리 딸이 엄마가 되더니 철이 들었네. 하지만 네가 웃어주면 세상 모든 시름이 사라졌단다.',
 '저는 너무 아무 준비 없이 아기를 가져서 앞으로 배워야 할 게 너무 많아요.',
 '공부하듯이 배우는 것이 아니라, 자연스레 익히게 될 거야. 너무 미리 걱정할 것 없어.',
 '아기를 낳고 나니 새로운 세상을 선물 받은 기분이 들어요. 기쁘고 설레고 걱정되고 두렵고.',
 '새로운 세상에 온 걸 환영한다, 내 딸. 엄마로서의 인생은 정말 선물 그 자체란다.',
 '앞으로 많이 가르쳐 주세요, 선배님.',
 '엄마 저 동해바다 보러 왔어요!',
 '이렇게 갑자기? 누구랑 같이 간 거야?',
 '아니요. 혼자 왔어요. 바람 좀 쏘이려고요.',
 '그랬구나. 요새 회사에서 무슨 일 있니? 말도 없이 갑자기 갔다고 하니 엄마가 조금 걱정이 되네.',
 '회사에선 아무 일 없어요. 일도 잘하고 있고요.',
 '그래. 엄마도 네가 일 똑 부러지게 하는 건 잘 알지. 그럼 무슨 다른 걱정거리라도 있니?',
 '아, 사실 저 남자 친구랑 얼마 전에 헤어졌거든요. 그래서 바닷바람 좀 쐴까 해서 왔어요.',
 '그랬구나. 그래도 둘이 일

In [8]:
y_train[:20]

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0]])

In [9]:
vaild_df['empathy'] = vaild_df['empathy'].apply(lambda x:list(map(int,str(x).split(','))))
mlb = MultiLabelBinarizer()
y_test = mlb.fit_transform(vaild_df['empathy'])
X_test = vaild_df['text'].tolist()

토크나이저 & 모델

In [10]:
!pip install accelerate peft

In [11]:
from accelerate import Accelerator
from peft import LoraConfig, TaskType, get_peft_model

tokenizer = BertTokenizerFast.from_pretrained('monologg/distilkobert')
model = DistilBertForSequenceClassification.from_pretrained('monologg/distilkobert', num_labels=6)

lora_config = LoraConfig(
    task_type = "SEQ_CLS", #시퀀스 분류 작업을 위한 설정
    inference_mode = False, #모델이 추론모드인지 여부 설정
    r=8, #낮은 랭크값을 사용하여 파라미터 수를 줄인다.
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=['attention.q_lin', 'attention.k_lin', 'attention.v_lin', 'attention.out_lin', 'ffn.lin1', 'ffn.lin2']
)

model = get_peft_model(model, lora_config)

accelerator = Accelerator() #데이터 병렬 처리 원활
model = accelerator.prepare(model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load fr

target_modules

- 'attention.q_lin',  # 쿼리 벡터를 생성하는 선형 레이어
- 'attention.k_lin',  # 키 벡터를 생성하는 선형 레이어
- 'attention.v_lin',  # 값 벡터를 생성하는 선형 레이어
- 'attention.out_lin',  # 어텐션 결과를 처리하는 출력 선형 레이어
- 'ffn.lin1',  # 첫 번째 FFN 선형 레이어
- 'ffn.lin2'  # 두 번째 FFN 선형 레이어

In [12]:
import torch
from torch.utils.data import Dataset, DataLoader

class MultiLabelDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.texts = texts
        self.labels = labels
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        labels = self.labels[index]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True,
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(labels, dtype=torch.float)
        }

max_len = 128
train_dataset = MultiLabelDataset(X_train, y_train, tokenizer, max_len)
vaild_dataset = MultiLabelDataset(X_test, y_test, tokenizer, max_len)

#train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

모델 훈련

In [13]:
sweep_config = {
    'method': 'random',  # 랜덤 탐색 방법
    'metric': {
        'name': 'eval_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 7e-5
        },
        'weight_decay': {
            'values': [0.1, 0.3, 0.5]
        },
        'batch_size': {
            'values': [8, 16, 32]
        },
        'epochs': {
            'values': [3]
        }
    }
}

In [14]:
!pip install evaluate

In [15]:
import evaluate
def compute_metrics(pred):
    logits, labels = pred
    predictions = (logits > 0.5).astype(int)  # 시그모이드 활성화 함수 적용 후 임계값 0.5 기준으로 이진화

    labels = labels.flatten()
    predictions = predictions.flatten()

    # 정확도, F1 score 계산
    acc_metrics = evaluate.load("accuracy")
    accuracy = acc_metrics.compute(predictions=predictions, references=labels)

    f1_metrics = evaluate.load("f1")
    f1 = f1_metrics.compute(predictions=predictions, references=labels)

    return {"accuracy": accuracy, "f1_metrics": f1}

In [19]:
from transformers import Trainer, TrainingArguments

# Sweep 생성
sweep_id = wandb.sweep(sweep_config, entity='sallylee', project='miniaiffelthon_DistilkoBERT_qlora_classification')

def train():
    with wandb.init() as run:
        config = wandb.config
        learning_rate = config.learning_rate
        batch_size = config.batch_size
        epochs = config.epochs

        # 데이터 로더 재정의 (학습 중 변경된 배치 크기를 반영)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        vaild_loader = DataLoader(vaild_dataset, batch_size=batch_size, shuffle=False)

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=epochs,
            per_device_train_batch_size=batch_size,
            per_device_eval_batch_size=batch_size,
            warmup_steps=500,
            weight_decay=0.01,
            logging_dir='./logs',
            logging_steps=10,
            evaluation_strategy="epoch"
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=vaild_dataset,
            compute_metrics=compute_metrics
        )

        trainer.train()
        eval_result = trainer.evaluate()

        # W&B에 평가 결과 기록
        wandb.log(eval_result)

def compute_metrics(p):
    pred, labels = p
    pred = (pred > 0.5).astype(int)
    accuracy = (pred == labels).mean()
    return {"accuracy": accuracy}

# Sweep 에이전트 실행
wandb.agent(sweep_id, train, count=1)

# W&B 세션 종료
wandb.finish()


Create sweep with ID: jhibgoat
Sweep URL: https://wandb.ai/sallylee/miniaiffelthon_DistilkoBERT_qlora_classification/sweeps/jhibgoat


wandb: Agent Starting Run: v4lm5ndz with config:
wandb: 	batch_size: 16
wandb: 	epochs: 3
wandb: 	learning_rate: 1.3849986782050944e-05
wandb: 	weight_decay: 0.3


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).


Epoch,Training Loss,Validation Loss,Accuracy
1,0.344300,0.352847,0.844760
2,0.332500,0.346737,0.847310
3,0.341300,0.343845,0.848823


epoch,▁
eval/accuracy,▁▅██
eval/loss,█▃▁▁
eval/runtime,▁▄█▄
eval/samples_per_second,█▅▁▅
eval/steps_per_second,█▅▁▅
eval_accuracy,▁
eval_loss,▁
eval_runtime,▁
eval_samples_per_second,▁
eval_steps_per_second,▁


In [20]:
model.save_pretrained('./DistilKoBERT-classification2')
tokenizer.save_pretrained('./DistilKoBERT-classification2')

('./DistilKoBERT-classification2/tokenizer_config.json',
 './DistilKoBERT-classification2/special_tokens_map.json',
 './DistilKoBERT-classification2/vocab.txt',
 './DistilKoBERT-classification2/added_tokens.json',
 './DistilKoBERT-classification2/tokenizer.json')

In [23]:
#저장된 모델,토크나이저 불러오기
model = DistilBertForSequenceClassification.from_pretrained('./DistilKoBERT-classification2',
                                              num_labels = 6,
                                              problem_type = "multi_label_classification")
trained_model = get_peft_model(model, lora_config)

tokenizer = BertTokenizerFast.from_pretrained('./DistilKoBERT-classification2')

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at monologg/distilkobert and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
import torch

def predict(text, model, tokenizer, max_len=128):
    model.eval()

    # 토크나이징
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=max_len,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
        truncation=True,
    )

    input_ids = encoding['input_ids']
    attention_mask = encoding['attention_mask']

    # 모델 예측
    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    probs = torch.sigmoid(logits).cpu().numpy()[0]
    predictions = (probs > 0.5).astype(int)

    return predictions, probs

In [25]:
# 예측할 텍스트
text = "나 시험을 못봐서 너무 슬퍼"

predictions, probs = predict(text, model, tokenizer)

print(f"Predictions: {predictions}")
print(f"Probabilities: {probs}")

Predictions: [1 1 0 1 1 0]
Probabilities: [0.5365158  0.51177144 0.46839547 0.514813   0.52337146 0.48636514]


In [26]:
text = "나 코딩이 너무 어려운 것 같아"

predictions, probs = predict(text, model, tokenizer)

print(f"Predictions: {predictions}")
print(f"Probabilities: {probs}")

Predictions: [1 1 0 1 1 0]
Probabilities: [0.54125476 0.5201753  0.46789747 0.51961017 0.5169131  0.48101327]


In [27]:
text = "나 너무 행복해"

predictions, probs = predict(text, model, tokenizer)

print(f"Predictions: {predictions}")
print(f"Probabilities: {probs}")

Predictions: [1 1 0 1 1 0]
Probabilities: [0.55124766 0.52876467 0.47038293 0.50335974 0.52974856 0.4904125 ]


In [28]:
text = "너 진짜 왜그러는거야!"

predictions, probs = predict(text, model, tokenizer)

print(f"Predictions: {predictions}")
print(f"Probabilities: {probs}")

Predictions: [1 1 0 1 1 1]
Probabilities: [0.5369501  0.52755356 0.48127705 0.5044319  0.5152815  0.5166452 ]
